# Retrofit cost tool: predictions with user data

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from retrofit_cost_tool import load_data, preprocess_data, predict, plot_predictions
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

In [2]:
# Create a file uploader widget
file_uploader = widgets.FileUpload(
    description='Upload data file',
    accept='.csv'
)

# Create a dropdown widget for model selection
model_selector = widgets.Dropdown(
    options=['ridge', 'elastic_net', 'random_forest', 'gradient_boosting', 'ols', 'glm_gamma', 'best_model'],
    value='best_model',
    description='Select model:',
    disabled=False
)

# Create a button widget to trigger prediction
predict_button = widgets.Button(description='Make Predictions')

# Create a button widget to trigger plotting
plot_button = widgets.Button(description='Plot Predictions')

# Create a checkbox widget to save plots
save_plots_checkbox = widgets.Checkbox(value=False, description='Save plots')

predictions_and_actuals = None

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output(wait=True)
        #print("Make predictions button clicked")
        global predictions_and_actuals
        file_name = file_uploader.value[0]['name']
        file_content = file_uploader.value[0]['content']
        with open(file_name, 'wb') as f:
            f.write(file_content.tobytes())
        data = load_data(file_name)
        target = list(data.columns)[-1]  # assume target is the last column
        predictions = predict(data, model_name=model_selector.value)
        actual_values = data[target]
        predictions_df = pd.DataFrame({'Predicted': predictions, 'Actual': actual_values})
        predictions_and_actuals = (predictions_df, actual_values)
        print(predictions_df.head())
        print("\nSummary Statistics:")
        print(predictions_df['Predicted'].describe())


def on_plot_button_click(b):
    global predictions_and_actuals
    if predictions_and_actuals is not None:
        predictions_df, actual_values = predictions_and_actuals
        with output:
            #output.clear_output(wait=True)
            plot_predictions(predictions_df, actual_values, save_plots=False)
    else:
        with output:
            output.clear_output(wait=True)
            print("Please make predictions first")


predict_button.on_click(on_button_click)
plot_button.on_click(on_plot_button_click)

In [3]:
display(file_uploader)
display(model_selector)
display(predict_button)
display(output)

FileUpload(value=(), accept='.csv', description='Upload data file')

Dropdown(description='Select model:', index=6, options=('ridge', 'elastic_net', 'random_forest', 'gradient_boo…

Button(description='Make Predictions', style=ButtonStyle())

Output()

In [4]:
display(plot_button)
display(save_plots_checkbox)

Button(description='Plot Predictions', style=ButtonStyle())

Checkbox(value=False, description='Save plots')